In [1]:
!gsutil -m cp -r gs://chunkbert_training/bert_model bert_model/

^C
Traceback (most recent call last):
  File "/usr/lib/google-cloud-sdk/platform/gsutil/gsutil", line 21, in <module>


In [2]:
!pip install transformers sentence_transformers
!pip install tokenizers
!pip install datasets
!pip install torch

  Using cached transformers-4.21.1-py3-none-any.whl (4.7 MB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached filelock-3.8.0-py3-none-any.whl (10 kB)
  Using cached tokenizers-0.12.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Using cached huggingface_hub-0.9.0-py3-none-any.whl (120 kB)
  Using cached regex-2022.8.17-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (752 kB)
  Using cached sentencepiece-0.1.97-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached torchvision-0.13.1-cp37-cp37m-manylinux1_x86_64.whl (19.1 MB)
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
  Using cached torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl (776.3 MB)
  Using cached datasets-2.4.0-py3-none-any.whl (365 kB)
  Using cached xxhash-3.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached multiprocess-0.70.13-py37-none-any.whl

In [3]:
from transformers import TFAutoModel

In [4]:
!transformers-cli convert --model_type bert --tf_checkpoint /home/jupyter/bert_model/model.ckpt --config /home/jupyter/bert_model/bert_config.json --pytorch_dump_output /home/jupyter/bert_model/pytorch_model.bin

Building PyTorch model from configuration: BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 128000
}

Converting TensorFlow checkpoint from /home/jupyter/bert_model/model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/beta/adam_m with shape [768]
Loading TF weight bert/embe

In [7]:
tf_model = TFAutoModel.from_pretrained("/home/jupyter/bert_model",from_pt=True)

2022-08-24 02:34:17.889198: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-24 02:34:18.573764: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-24 02:34:18.574562: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-24 02:34:18.586738: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [8]:
type(tf_model)

transformers.models.bert.modeling_tf_bert.TFBertModel

In [9]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("/home/jupyter/bert_model", local_files_only=True,truncation=True,padding=True)

In [10]:
tf_model.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  184345344 
                                                                 
Total params: 184,345,344
Trainable params: 184,345,344
Non-trainable params: 0
_________________________________________________________________


In [58]:
!gsutil -m cp -r gs://chunkerbert/v2/files /home/jupyter/bert_model_linux2/testfiles/

Copying gs://chunkerbert/v2/files/Dockerfile.txt...
Copying gs://chunkerbert/v2/files/bertchunker.exe.txt...                        
Copying gs://chunkerbert/v2/files/bertchunker.txt...                            
Copying gs://chunkerbert/v2/files/chunker.go.txt...                             
/ [4 files][ 36.9 KiB/ 36.9 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://chunkerbert/v2/files/cloudbuild.yaml.txt...
Copying gs://chunkerbert/v2/files/ls.txt...                                     
Copying gs://chunkerbert/v2/files/pom.xml.txt...                                
Copying gs://chunkerbert/v2/files/pool0-volume-cfg.xml.txt...                   
Copying gs://chunkerbert/v2/files/ssh.txt...                            

In [11]:
from os import listdir
from os.path import isfile, join
flp = '/home/jupyter/bert_model_linux2/testfiles/files'
onlyfiles = [f for f in listdir(flp) if isfile(join(flp, f))]
lines = []
fnms = []
for file in onlyfiles:
    with open(join(flp, file)) as f:
        lns = f.readlines()
        i=0
        for line in lns:
            lines.append(line)
            fnms.append("{fnm}-{ln}".format(fnm=file,ln=i))
            i+=1



In [12]:
len(lines)

26

In [61]:
import tensorflow as tf
#input_ids = tokenizer.encode(lines,padding=True, max_length=512, truncation=True,return_tensors='tf')  # Batch size 1
input_ids=tokenizer(lines, return_tensors='tf',padding=True, max_length=512, truncation=True)
outputs = tf_model(input_ids)
last_hidden_states = outputs[0]
pooled_states=outputs[1]

In [63]:
last_hidden_states[0].shape
pooled_states.shape

TensorShape([26, 768])

In [76]:
output = []
comp = "bertchunker.exe.txt-0"
for i in range(len(lines)):
    output.append(pooled_states[i])
    if fnms[i] == comp:
        target = pooled_states[i]

    
    

In [77]:
from sklearn.metrics.pairwise import cosine_similarity
sim = cosine_similarity(
    [target],
    output[0:]
)
for i in range(len(output)):
    print("{fnm} = {opt}".format(fnm=fnms[i],opt=sim[0][i]))

ssleay32.dll.txt-0 = 0.977300763130188
Dockerfile.txt-0 = 0.527336061000824
pom.xml.txt-0 = 0.7544726729393005
cloudbuild.yaml.txt-0 = 0.7413095235824585
chunker.go.txt-0 = 0.6230233311653137
ssh.txt-0 = 0.9137042164802551
bertchunker.txt-0 = 0.9522317051887512
bertchunker.txt-1 = 0.9215943813323975
bertchunker.txt-2 = 0.9230149388313293
bertchunker.txt-3 = 0.9410337805747986
bertchunker.txt-4 = 0.9356136322021484
bertchunker.txt-5 = 0.963344931602478
bertchunker.txt-6 = 0.9762070775032043
bertchunker.txt-7 = 0.9870809316635132
bertchunker.txt-8 = 0.7836321592330933
ls.txt-0 = 0.8440443873405457
bertchunker.exe.txt-0 = 1.000000238418579
bertchunker.exe.txt-1 = 0.981987714767456
bertchunker.exe.txt-2 = 0.9564316272735596
bertchunker.exe.txt-3 = 0.966778039932251
bertchunker.exe.txt-4 = 0.9846380949020386
bertchunker.exe.txt-5 = 0.9879868626594543
bertchunker.exe.txt-6 = 0.9743245840072632
bertchunker.exe.txt-7 = 0.9245784878730774
bertchunker.exe.txt-8 = 0.9794593453407288
pool0-volume-

In [72]:
output = []
comp = "pool0-volume-cfg.xml.txt-0"
for i in range(len(lines)):
    encoded_input = tokenizer(lines[i], return_tensors='tf',padding=True, max_length=512, truncation=True)
    output.append(tf_model(encoded_input)[1][0])
    if fnms[i] == comp:
        target = tf_model(encoded_input)[1][0]


In [73]:
from sklearn.metrics.pairwise import cosine_similarity
sim = cosine_similarity(
    [target],
    output[0:]
)
for i in range(len(output)):
    print("{fnm} = {opt}".format(fnm=fnms[i],opt=sim[0][i]))

ssleay32.dll.txt-0 = 0.6615363955497742
Dockerfile.txt-0 = 0.8694315552711487
pom.xml.txt-0 = 0.9509569406509399
cloudbuild.yaml.txt-0 = 0.9674720764160156
chunker.go.txt-0 = 0.9010655879974365
ssh.txt-0 = 0.7981199026107788
bertchunker.txt-0 = 0.620159924030304
bertchunker.txt-1 = 0.5780822038650513
bertchunker.txt-2 = 0.5776832103729248
bertchunker.txt-3 = 0.6012235283851624
bertchunker.txt-4 = 0.5951313972473145
bertchunker.txt-5 = 0.6279081702232361
bertchunker.txt-6 = 0.6385867595672607
bertchunker.txt-7 = 0.6654609441757202
bertchunker.txt-8 = 0.8566379547119141
ls.txt-0 = 0.5386608839035034
bertchunker.exe.txt-0 = 0.6981604099273682
bertchunker.exe.txt-1 = 0.6485766172409058
bertchunker.exe.txt-2 = 0.6121979355812073
bertchunker.exe.txt-3 = 0.636157751083374
bertchunker.exe.txt-4 = 0.6717057228088379
bertchunker.exe.txt-5 = 0.7372808456420898
bertchunker.exe.txt-6 = 0.6489080190658569
bertchunker.exe.txt-7 = 0.7967387437820435
bertchunker.exe.txt-8 = 0.753321647644043
pool0-volu

In [32]:
comp = "bertchunker.txt-0"
output = []
ec = tokenizer(lines, return_tensors='tf',padding=True, max_length=512, truncation=True)
for i in range(len(lines)):
    op = tf_model(ec[i])[1][0]
    output.append(op)
    if fnms[i] == comp:
        target = tf_model(op)

KeyError: 'Indexing with integers (to access backend Encoding for a given batch index) is not available when using Python based tokenizers'

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
sim = cosine_similarity(
    [target],
    output[0:]
)
for i in range(len(output)):
    print("{fnm} = {opt}".format(fnm=fnms[i],opt=sim[0][i]))
    

ssleay32.dll.txt-0 = 0.9805154800415039
Dockerfile.txt-0 = 0.36777105927467346
pom.xml.txt-0 = 0.617067813873291
cloudbuild.yaml.txt-0 = 0.37110263109207153
chunker.go.txt-0 = 0.7078962326049805
ssh.txt-0 = 0.9180433750152588
bertchunker.txt-0 = 1.0
bertchunker.txt-1 = 0.9082966446876526
bertchunker.txt-2 = 0.9755203723907471
bertchunker.txt-3 = 0.9378695487976074
bertchunker.txt-4 = 0.9725769758224487
bertchunker.txt-5 = 0.9740099906921387
bertchunker.txt-6 = 0.9068374037742615
bertchunker.txt-7 = 0.9888618588447571
bertchunker.txt-8 = 0.7374651432037354
ls.txt-0 = 0.896312952041626
bertchunker.exe.txt-0 = 0.9768720865249634
bertchunker.exe.txt-1 = 0.9510067701339722
bertchunker.exe.txt-2 = 0.969919741153717
bertchunker.exe.txt-3 = 0.9817095994949341
bertchunker.exe.txt-4 = 0.9811372756958008
bertchunker.exe.txt-5 = 0.9421320557594299
bertchunker.exe.txt-6 = 0.9834581613540649
bertchunker.exe.txt-7 = 0.9693260192871094
bertchunker.exe.txt-8 = 0.8724168539047241
pool0-volume-cfg.xml.tx

In [39]:
output[1]

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[ 0.98356175, -0.763272  , -0.973702  , -0.9746375 ,  0.9987762 ,
         0.98092973, -0.91556203, -0.99628425,  0.85820407, -0.9332739 ,
         0.999987  , -0.98802626,  0.78244424, -0.82041734, -0.9796203 ,
         0.9957244 , -0.95424086, -0.905965  , -0.97832584,  0.9925657 ,
        -0.9824898 ,  0.9905087 ,  0.95914686,  0.9820703 , -0.99852943,
        -0.98136336,  0.99654984,  0.18576406,  0.92985255,  0.9945827 ,
        -0.9461659 , -0.96482414,  0.99833435,  0.96462727, -0.89341545,
         0.9523821 , -0.9916613 , -0.91113883,  0.06847017,  0.9974047 ,
         0.37909704,  0.95211726, -0.89083487, -0.9620681 ,  0.8607059 ,
        -0.9957512 ,  0.9989624 , -0.9954109 , -0.9951788 , -0.9896972 ,
         0.8071308 , -0.9943809 , -0.9944004 ,  0.97707444, -0.99817973,
         0.9986314 , -0.995197  ,  0.95258826,  0.9770832 ,  0.9193328 ,
        -0.99955434, -0.98400193,  0.99255997, -0.96371543,  0.9863481 ,
 